# Loader for Space Divers Mini sound lib

In [1]:
SOURCE_NAME = "space_divers_mini"
ORIGINAL_PATH = "./data/original/" + SOURCE_NAME + "/"
SAMPLES_PATH = ORIGINAL_PATH + "Samples/"
TARGET_PATH = "./data/intermediate/" + SOURCE_NAME + "/"

## Get the filenames to parse

In [2]:
import os

In [3]:
def load_filenames(path: str) -> list[str]:
    "Load a list of audio filenames in the directory."
    return [filename for filename in os.listdir(path) if filename.endswith(".wav")]

In [4]:
filenames = load_filenames(SAMPLES_PATH)
filenames[:5]

['99S LT Ambience Underwater - Coral Sea.wav',
 '99S LT Manipulate - Distant.wav',
 '99S LT Impact - Crash Distant A.wav',
 '99S LT Underwater - Move D.wav',
 '99S LT Vehicle - Resonation Slow.wav']

## Parse the files into labels

The titles follow a couple of patterns but follow within 2:  
`<space separated categories> - <specific label>.wav`  
`<specific label>.wav`  

Space separated categories can also include the `-` split.  
The specific labels can have `A B C...` variants which get ignored.

In [22]:
import string
import stringcase

In [35]:
def parse_filename(filename: str) -> list[str]:
    "Parse the filename to get a list of labels."
    # Remove the prefix and extension
    title = filename.removeprefix("99S LT ").removesuffix(".wav")
    *labels, specific_label = title.split("-")
    labels = [l.strip() for label in labels for l in label.split()]
    # Remove the variant at the end
    specific_label = specific_label.rstrip(string.whitespace + string.ascii_uppercase)
    # We have to l.lower() first otherwise stringcase.snakecase will prepend multiple underscores
    normalise = lambda x: stringcase.capitalcase(stringcase.snakecase(x.lower().strip()))
    return [normalise(l) for l in labels + [specific_label]]

In [36]:
labels = [parse_filename(filename) for filename in filenames]
labels[:5]

[['Ambience', 'Underwater', 'Coral_sea'],
 ['Manipulate', 'Distant'],
 ['Impact', 'Crash_distant'],
 ['Underwater', 'Move'],
 ['Vehicle', 'Resonation_slow']]

## Copy the files to the intermediate folder

In [14]:
import xxhash
import shutil

In [11]:
def get_hash(filename: str) -> str:
    "Get the sha2 hash of the file."
    with open(os.path.normpath(filename), "rb") as f:
        return xxhash.xxh64(f.read()).hexdigest()

In [12]:
file_hashes = [get_hash(SAMPLES_PATH + filename) for filename in filenames]
file_hashes[:5]

['df329b2f45088143',
 'beda557bed5629a2',
 '56b3dae6f6efd75f',
 'baaddb69127f8753',
 '6c110836db6654e5']

In [15]:
if not os.path.exists(TARGET_PATH):
    os.makedirs(TARGET_PATH)

# Copy the files to the target directory
for filename, hash in zip(filenames, file_hashes):
    shutil.copyfile(SAMPLES_PATH + filename, TARGET_PATH + hash + ".wav")

## Export the CSV rows

In [33]:
import csv

In [38]:
# Create a CSV file with the labels
source = [SOURCE_NAME] * len(labels)
with open(TARGET_PATH + "metadata.csv", "w") as f:
    writer = csv.writer(f)
    writer.writerow(["filename", "label", "source"])
    writer.writerows(zip(file_hashes, [",".join(l) for l in labels], source))